In [3]:
dadagp_path = '/Users/pedro/Desktop/DadaGP-v1.1/'
all_files_path = '/Users/pedro/Desktop/DadaGP-v1.1/_DadaGP_all_filenames.json'

## Get all .gp files

In [12]:
import json

# Assuming 'all_files_path' contains the path to your JSON file
with open(all_files_path, 'r') as f:
    file_paths = json.load(f)  # Load JSON data into a Python list

# Remove the ".tokens.txt" part from each path
all_gps_paths = [path.replace(".tokens.txt", "") for path in file_paths]

# Print the number of files
print(len(all_gps_paths))

26181


## Get all .gp files with => 4 tracks

In [25]:
import guitarpro as gp
import os
from tqdm import tqdm

gp_files_4plus = []

# Iterate over the list of .gp file paths
for gp_file_path in tqdm(all_gps_paths):
    # Get full path
    file_path = os.path.join(dadagp_path, gp_file_path)
    # Parse them with pyguitarpro
    try:
        # Parse the Guitar Pro file
        gp_file = gp.parse(file_path)

        # Check the number of tracks
        if len(gp_file.tracks) >= 4:
            gp_files_4plus.append(file_path)
            # print('File {} has {} tracks.'.format(file_path, len(gp_file.tracks)))

    except Exception as e:
        # Handle any parsing errors gracefully
        print('Could not parse {}: {}'.format(file_path, e))
    

  2%|▏         | 588/26181 [00:45<26:02, 16.38it/s]  